In [1]:
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install keras
!pip install tensorflow
!pip install pandas
!pip install opencv-python



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\mazen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from os import getcwd

# Load the CSV file
dataset_dir = getcwd() + '/full_dataset'
csv_file = 'labels.csv'

df = pd.read_csv(dataset_dir + '/' + csv_file)

# Function to load images
def load_image(row):
    image_path = f"{dataset_dir}/{row['filename']}"
    image = cv2.imread(image_path)

    return image

def normalize_image(data):
    data = cv2.normalize(data, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    return data

In [3]:
# Apply the function to each row in the DataFrame
df['normalized_image'] = df.apply(load_image, axis=1)

In [4]:
def resize_image(data):
    if data is not None and data.any() and data.size != 0:
        new_height, new_width = 416, 416
        data = cv2.resize(data, (new_width, new_height))
    
    return data

In [5]:
x_data = []
labels = []

# Resize images & filter NAN classes
for i in range(0, len(df['normalized_image'])):
    if df['normalized_image'][i] is not None and df['normalized_image'][i].any():
        x_data.append(resize_image(df['normalized_image'][i]))
        labels.append(df['class'][i])

In [6]:
# Normalize images
x_data = [normalize_image(x) for x in x_data]

In [7]:
y_df = pd.DataFrame({'class': labels})

In [8]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Use LabelEncoder to convert string labels to numerical labels
encoder = LabelEncoder()
y_data = encoder.fit_transform(y_df['class'])

# Convert numerical labels to one-hot encoding
y_data = to_categorical(y_data)

# Feed Forward Neural Network

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

def create_multilayer_model(learning_rate, num_hidden_nodes, num_hidden_layers, 
                            activation_fn = 'sigmoid', init_method= 'random', 
                            num_classes = 12):
    model = Sequential()
    
    # Input layer
    model.add(Flatten(input_shape=(416, 416, 3)))

    # Hidden layers
    for i in range(num_hidden_layers):
        if init_method == 'constant':
            model.add(Dense(num_hidden_nodes, activation=activation_fn, kernel_initializer=Constant(0.0000009)))
        
        else:
            model.add(Dense(num_hidden_nodes, activation=activation_fn, kernel_initializer='random_uniform'))
    
    # Output layer
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
# Trimmed data to 500 images to avoid memory errors
x_data_trim = x_data[:500]
y_data_trim = y_data[:500]

In [11]:
from sklearn.model_selection import train_test_split

x_data_trim = np.array(x_data_trim)

x_train, x_test, y_train, y_test = train_test_split(x_data_trim, y_data_trim, test_size=0.2, random_state=42)

In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# Initialize Kfold crossvalidation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# define hyperparameters
epochs = 30
learning_rate = 0.03
num_hidden_nodes = 40
layers = 1
batch_size = 200

models = []
i = 0

for train_index, val_index in kf.split(x_train):
    # split train into train and validation sets
    x_train_fold = x_train[train_index] 
    x_val_fold = x_train[val_index]
    y_train_fold = y_train[train_index]  
    y_val_fold = y_train[val_index]

    # Create & train the model
    model = create_multilayer_model(learning_rate, num_hidden_nodes, layers, 'relu')
    model.fit(x_train_fold, y_train_fold, batch_size=batch_size, epochs=epochs, validation_data=(x_val_fold, y_val_fold), verbose=0)

    # Save the model
    i += 1 
    model.save(f'model_fold_{i + 1}.h5')
    models.append(f'model_fold_{i + 1}.h5')
    
    # Calculate train metrics
    train_loss, train_accuracy = model.evaluate(x_train_fold, y_train_fold, verbose=0)
    train_predictions = model.predict(x_train_fold)
    train_f1_score = f1_score(np.argmax(y_train_fold, axis=1), np.argmax(train_predictions, axis=1), average='macro') 

    # Calculate validation metrics
    val_loss, val_accuracy = model.evaluate(x_val_fold, y_val_fold, verbose=0)
    val_predictions = model.predict(x_val_fold)
    val_f1_score = f1_score(np.argmax(y_val_fold, axis=1), np.argmax(val_predictions, axis=1), average='macro') 

    # Calculate test metrics
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    test_predictions = model.predict(x_test)
    test_f1_score = f1_score(np.argmax(y_test, axis=1), np.argmax(test_predictions, axis=1), average='macro') 

    print(train_accuracy, train_f1_score)
    print(val_accuracy, val_f1_score)
    print(test_accuracy, test_f1_score)


C:\Users\mazen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 13ms/step
0.5406249761581421 0.2339418526031102
0.5625 0.24
0.46000000834465027 0.21004566210045664


C:\Users\mazen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 12ms/step
0.534375011920929 0.23217922606924646
0.5874999761581421 0.24671916010498687
0.4399999976158142 0.1527777777777778


C:\Users\mazen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 12ms/step
0.550000011920929 0.17777777777777778
0.5249999761581421 0.2295081967213115
0.46000000834465027 0.21004566210045664


C:\Users\mazen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 12ms/step
0.543749988079071 0.23481781376518218
0.550000011920929 0.23655913978494625
0.46000000834465027 0.21004566210045664


C:\Users\mazen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 12ms/step
0.5625 0.2577486577486577
0.5 0.16806722689075632
0.44999998807907104 0.20833333333333334


In [13]:
print(models)

['model_fold_2.h5', 'model_fold_3.h5', 'model_fold_4.h5', 'model_fold_5.h5', 'model_fold_6.h5']
